# Day 4 Lecture

In [1]:
import math

def calculate_LJ(r_ij):
    r6_term = math.pow(1/r_ij, 6)
    r12_term = math.pow(r6_term, 2)
    pairwise_energy = 4 * (r12_term - r6_term)
    return pairwise_energy


def calculate_distance(coord1, coord2, box_length=None):
    """
    Calculate the distance between two 3D coordinates.

    Parameters
    ----------
    coord1, coord2: list
        The atomic coordinates

    Returns
    -------
    distance: float
        The distance between the two points.
    """

    distance = 0
    for i in range(3):
        dim_dist = coord1[i] - coord2[i]

        if box_length:
            dim_dist = dim_dist - box_length * round(dim_dist / box_length)

        dim_dist = dim_dist**2
        distance += dim_dist

    distance = math.sqrt(distance)
    return distance
    
def calculate_total_energy(coordinates, box_length, cutoff):
    """
    Calculate the total Lennard Jones energy of a system of particles.

    Parameters
    ----------
    coordinates : list
        Nested list containing particle coordinates.

    Returns
    -------
    total_energy : float
        The total pairwise Lennard Jones energy of the system of particles.
    """

    total_energy = 0

    num_atoms = len(coordinates)

    for i in range(num_atoms):
        for j in range(i + 1, num_atoms):

            dist_ij = calculate_distance(
                coordinates[i], coordinates[j], box_length=box_length
            )

            if dist_ij < cutoff:
                interaction_energy = calculate_LJ(dist_ij)
                total_energy += interaction_energy

    return total_energy

def read_xyz(filepath):
    """
    Reads coordinates from an xyz file.
    
    Parameters
    ----------
    filepath : str
       The path to the xyz file to be processed.
       
    Returns
    -------
    atomic_coordinates : list
        A two dimensional list containing atomic coordinates

    """
    
    with open(filepath) as f:
        box_length = float(f.readline().split()[0])
        num_atoms = float(f.readline())
        coordinates = f.readlines()
    
    atomic_coordinates = []
    
    for atom in coordinates:
        split_atoms = atom.split()
        
        float_coords = []
        
        # We split this way to get rid of the atom label.
        for coord in split_atoms[1:]:
            float_coords.append(float(coord))
            
        atomic_coordinates.append(float_coords)
        
    
    return atomic_coordinates, box_length

def calculate_tail_correction(num_particles, box_length, cutoff):
    """
    Calculate the long range tail correction
    """

    const1 = (8 * math.pi * num_particles**2) / (3 * box_length**3)
    const2 = (1 / 3) * (1 / cutoff) ** 9 - (1 / cutoff) ** 3

    return const1 * const2

In [2]:
nist_3cut = -4.3515E+03
nist_4cut = -4.4675E+03

config1_file = "../data/sample_config1.txt"

coords, box_length = read_xyz(config1_file)

calc_3cut = calculate_total_energy(coords, box_length, 3)
calc_4cut = calculate_total_energy(coords, box_length, 4)

In [3]:
print(f"Cut off 3 energy: {calc_3cut}")
print(f"Cut off 4 energy: {calc_4cut}")

Cut off 3 energy: -4351.540194543858
Cut off 4 energy: -4467.495724947969


In [4]:
assert math.isclose(calc_3cut, nist_3cut, rel_tol=0.02)
assert math.isclose(calc_4cut, nist_4cut, rel_tol=0.02)

In [5]:
import random 

def accept_or_reject(delta_e, beta):
    """
    Accept or reject based on an energy and beta (inverse temperature) measurement.

    Parameters
    ----------
    delta_e: float
        An energy change in reduced units.
    beta: float
        Inverse temperature

    Returns
    -------
    bool
        True to accept move, False to reject
    """

    if delta_e <= 0.0:
        accept = True
    else:
        random_number = random.random()
        p_acc = math.exp(-delta_e*beta)

        if random_number < p_acc:
            accept = True
        else:
            accept = False
    
    return accept

In [6]:
# Sanity check
delta_energy = -1
beta = 1
assert accept_or_reject(delta_energy, beta) is True

In [7]:
delta_energy = 0
beta = 1
assert accept_or_reject(delta_energy, beta) is True

In [8]:
delta_energy = 1
beta = 1
accept_or_reject(delta_energy, beta)

False

In [9]:
random.seed(0)
delta_energy = 1
beta = 1
assert accept_or_reject(delta_energy, beta) is False

In [10]:
p_acc = math.exp(-1 * 1)
print(p_acc)

0.36787944117144233


In [11]:
# Find a random seed that will result in the move being accepted.
# We're looking for a random seed where the generated number is less than 0.36787944117144233.
random.seed(1)
assert accept_or_reject(delta_energy, beta) is True

In [12]:
def calculate_pair_energy(coordinates, i_particle, box_length, cutoff):
    """
    Calculate the interaction energy of a particle with its environment (all other particles in the system).

    Parameters
    ----------
    coordinates: list
        The coordinates for all particles in the system.
    i_particle: int
        The particle index for which to calculate the energy.
    cutoff: float
        The simulation cutoff. Beyond this distance, the interactions are not calculated.

    Returns
    -------
    e_total: float
        The pairwise interaction energy of the i_th particle with all other particles.
    """

    e_total = 0.0
    i_position = coordinates[i_particle]

    num_atoms = len(coordinates)

    for j_particle in range(num_atoms):
        if i_particle != j_particle:

            j_position = coordinates[j_particle]

            r_ij = calculate_distance(i_position, j_position, box_length)

            if r_ij < cutoff:
                e_pair = calculate_LJ(r_ij)
                e_total += e_pair

    return e_total

In [13]:
atom1 = [0, -math.pow(2, (1/6)), 0]
atom2 = [0, 0, 0]
atom3 = [0, math.pow(2, (1/6)), 0]

coordinates = [ atom1, atom2, atom3 ]
print(coordinates)

[[0, -1.122462048309373, 0], [0, 0, 0], [0, 1.122462048309373, 0]]


In [14]:
assert calculate_pair_energy(coordinates, 1, 10, 3) == -2.0
assert calculate_pair_energy(coordinates, 2, 10, 3) == calculate_pair_energy(coordinates, 0, 10, 3)

## Simulation Loop

In [29]:
# If you want your results to be reproducible
random.seed(0)

# Simulation Parameters
reduced_temperature = 0.9
num_steps = 5000
max_displacement = 0.1
cutoff = 3.0

freq = 1000

# Calculated quantites
beta = 1 / reduced_temperature

# Get initial coordinates (initial system configuration)
coordinates, box_length = read_xyz("../data/sample_config1.txt")
num_particles = len(coordinates)

delta_energy = 0

total_energy = calculate_total_energy(coordinates, box_length, cutoff)
total_energy += calculate_tail_correction(num_particles, box_length, cutoff)

In [30]:
steps = []
energies = []

for step in range(num_steps):

    # 1. Randomly pick one of N particles
    random_particle = random.randrange(num_particles)

    # 2. Calculate the interaction energy of the selected particle with the system and store this value.
    current_energy = calculate_pair_energy(coordinates, random_particle, box_length, cutoff) 

    # 3. Generate a random x, y, z displacement
    x_rand = random.uniform(-max_displacement, max_displacement) 
    y_rand = random.uniform(-max_displacement, max_displacement) 
    z_rand = random.uniform(-max_displacement, max_displacement) 

    # 4. Modify the coordinate of Nth particle by the generated displacements.
    coordinates[random_particle][0] += x_rand
    coordinates[random_particle][1] += y_rand
    coordinates[random_particle][2] += z_rand

    # 5. Calculate the interaction energy of the selected particle with the system and store this value (using the new position).
    proposed_energy = calculate_pair_energy(coordinates, random_particle, box_length, cutoff) 

    # 6. Calculate the change in energy and decide to accept or reject the change.
    delta_e = proposed_energy - current_energy

    accept = accept_or_reject(delta_e, beta)

    # 7. If accept, keep particle movement. If reject, move particle back.
    if accept:
        total_energy += delta_e
    else:
        # Move is not accepted, roll back coordiantes
        coordinates[random_particle][0] -= x_rand
        coordinates[random_particle][1] -= y_rand
        coordinates[random_particle][2] -= z_rand


    # 8. Print the energy if the step is a multiple of freq
    if step % freq == 0:
        print(step, total_energy/num_particles)
        steps.append(step)
        energies.append(total_energy)

0 -5.6886200705459125
1000 -5.679007963444937
2000 -5.699453759871805
3000 -5.654305513583807
4000 -5.646748359462866
